In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [2]:
from azureml.core.model import Model

model1 = ws.models['Fraud_Detect_Model']

In [3]:
import os

folder_name = './'
script_file = os.path.join(folder_name,"score_script.py")

In [4]:
%%writefile $script_file
import json
import os
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
import joblib

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType

def init():
    global model_pipe
    model_pipe = joblib.load(os.path.join(os.getenv("AZUREML_MODEL_DIR"), "PipeFraudDetection.pkl"))


input_sample = pd.DataFrame(data = [{'id': 45, 'date' : "abc", 'amount': 95.55, 'old_balance': 142590.743, 
          'income': 2979.713, 'credit': 0, 'marital_status': 1, 'children': 2, 
          'month': 10, 'day_of_week': 'monday', 'category_recipient': 'food_store', 
          'transaction_type': 'onsite', 'transaction_method': 'other', 'foreign_transaction': 1}])
output_sample = np.array([1.0])

@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))

def run(data):
    data.drop(['id','date'], axis=1, inplace = True)
    pred = model_pipe.predict(data)
    return pred.tolist()

Overwriting ./score_script.py


In [5]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn=0.22.1')
myenv.add_conda_package('xgboost==1.2.0')
myenv.add_conda_package('pandas==1.0.3')
myenv.add_conda_package('numpy')
myenv.add_conda_package('joblib')
myenv.add_pip_package('inference-schema')

env_file = os.path.join(folder_name,"scoring_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
    
env_file = os.path.join(folder_name,"scoring_env.yml")

In [6]:
from azureml.core.compute import AksCompute

aks_target = AksCompute(ws,"My-Kub-Inf")

In [7]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import AksWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Model

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 0.1, memory_gb = 0.5, auth_enabled = False)

service_name = "rt-fd-ml-scoring"

service = Model.deploy(ws, service_name, [model1], inference_config, deployment_config, aks_target)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-03-18 09:48:16+00:00 Creating Container Registry if not exists.
2022-03-18 09:48:16+00:00 Registering the environment.
2022-03-18 09:48:18+00:00 Use the existing image.
2022-03-18 09:48:18+00:00 Generating deployment configuration.
2022-03-18 09:48:19+00:00 Submitting deployment to compute.
2022-03-18 09:48:27+00:00 Checking the status of deployment rt-fd-ml-scoring..
2022-03-18 09:51:47+00:00 Checking the status of inference endpoint rt-fd-ml-scoring.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [8]:
print(service.state)

Healthy


In [9]:
import json

x_new = {
    "data": [{'id': 4, 'date' : 'yo', 'amount': 95.55648880442871, 'old_balance': 142590.44432050743, 
          'income': 2979.7136940079527, 'credit': 0, 'marital_status': 1, 'children': 2, 
          'month': 10, 'day_of_week': 'monday', 'category_recipient': 'food_store', 
          'transaction_type': 'onsite', 'transaction_method': 'other', 'foreign_transaction': 1}
    ]
}

input_json = json.dumps(x_new)

predictions = service.run(input_data = input_json)

print(predictions)

[0]


In [ ]:
endpoint = service.scoring_uri
testuri = service.swagger_uri
print(testuri)
print(endpoint)

In [ ]:
#token, refresh_by = service.get_keys()
#print(token)

In [ ]:
import requests
import json

x_new = {'id': 10000, 'amount': 95.55648880442871, 'old_balance': 142590.44432050743, 
          'income': 2979.7136940079527, 'credit': 0, 'marital_status': 1, 'children': 2, 
          'month': 10, 'day_of_week': 'monday', 'category_recipient': 'food_store', 
          'transaction_type': 'onsite', 'transaction_method': 'other', 'foreign_transaction': 1}

input_json = json.dumps({"data": x_new})

headers = { 'Content-Type':'application/json', 'Authorization' : 'Bearer' + token}

predictions = requests.post(endpoint, input_json, headers = headers)
print(predictions.json())



In [ ]:
service = ws.webservices['test-scoring-service']
print(service)

In [ ]:
service.delete()

In [ ]:
WITH 
Dataframe AS (
SELECT id, amount, old_balance, income, credit, marital_status, children, month, day_of_week, category_recipient, transaction_type, transaction_method, foreign_transaction
FROM input
)

SELECT id, amount, old_balance, income, credit, marital_status, children, month, day_of_week, category_recipient, transaction_type, transaction_method, foreign_transaction, udf.prediction(Dataframe)
INTO output
FROM Dataframe